In [1]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
import scipy.signal


In [87]:
L = 100
W = 100
D = int(0.1*L*W)

In [88]:
grid = np.zeros((L, W), dtype=np.bool)
grid[np.random.randint(L, size=D), np.random.randint(W, size=D)] = 1
#grid[:5,:] = 1
#grid [10:100, 30:50]= 1
#grid [100:400, 100:400]= 1


fig, ax = plt.subplots(figsize=(10, 10))
gr = ax.imshow(grid, cmap='gray')


def count_neighbors(grid, i, j):
    return sum(map(int, [grid[a%W, b%L] 
                         for a in range(i-1, i+2) 
                         for b in range(j-1, j+2) 
                         if (a%W!=i or b%L!=j)]))

def compute_next_grid(grid):
    new_grid = grid.copy()
    all_neighbors = scipy.signal.convolve2d(grid, np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]]), 
                                             mode='same', boundary='wrap')
    
    new_grid += np.logical_not(grid)*(all_neighbors==3)
    new_grid ^= grid*(all_neighbors < 2) + grid*(all_neighbors > 3)
    return new_grid

def animate(t, grid):
    

    grid[:] = compute_next_grid(grid)[:]
    gr.set_data(grid)
    
ani = matplotlib.animation.FuncAnimation(fig, animate, fargs=(grid,), blit=False, interval=0)
ax.axis('off')
fig.tight_layout()
fig.show()

In [66]:
from keras.layers import Conv2D, ReLU, Dense
from keras.models import Sequential
from keras.losses import mean_squared_error
from keras.optimizers import Adadelta
import keras

In [89]:
X_train = []
Y_train = []

for _ in range(10):
    grid = np.zeros((L, W), dtype=np.bool)
    grid[np.random.randint(L, size=D), np.random.randint(W, size=D)] = 1
    X_train.append(grid)
    for _ in range(1000):
        new_grid = compute_next_grid(grid)
        Y_train.append(new_grid[:])
        X_train.append(new_grid[:])
        
    new_grid = compute_next_grid(grid)
    Y_train.append(new_grid[:])
    
X_val = []
Y_val = []

for _ in range(10):
    grid = np.zeros((L, W), dtype=np.bool)
    grid[np.random.randint(L, size=D), np.random.randint(W, size=D)] = 1
    X_val.append(grid)
    for _ in range(200):
        new_grid = compute_next_grid(grid)
        X_val.append(new_grid[:])
        Y_val.append(new_grid[:])
        
    new_grid = compute_next_grid(grid)
    Y_val.append(new_grid[:])

In [90]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_val = np.array(X_val)
Y_val = np.array(Y_val)

In [126]:
m = Sequential()
m.add(Conv2D(filters=2, kernel_size=3, input_shape=(100, 100, 1), padding="same", activation="relu"))
m.add(Dense(1, activation='tanh'))

m.compile(loss=mean_squared_error,
          optimizer=Adadelta(), 
          metrics=[keras.metrics.mae])

In [127]:
m.fit(X_train[:, :, :, np.newaxis], Y_train[:, :, :, np.newaxis], 
      epochs=5, batch_size=32, validation_data=[X_val[:, :, :, np.newaxis], Y_val[:, :, :, np.newaxis]])

Train on 10010 samples, validate on 2010 samples
Epoch 1/5
10010/10010 [==============================] - 17s 2ms/step - loss: 0.0338 - mean_absolute_error: 0.0674 - val_loss: 0.0058 - val_mean_absolute_error: 0.0213
Epoch 2/5
10010/10010 [==============================] - 17s 2ms/step - loss: 0.0013 - mean_absolute_error: 0.0094 - val_loss: 6.4927e-04 - val_mean_absolute_error: 0.0060
Epoch 3/5
10010/10010 [==============================] - 18s 2ms/step - loss: 2.0322e-04 - mean_absolute_error: 0.0053 - val_loss: 5.5298e-04 - val_mean_absolute_error: 0.0058
Epoch 4/5
10010/10010 [==============================] - 20s 2ms/step - loss: 1.5449e-04 - mean_absolute_error: 0.0057 - val_loss: 5.3579e-04 - val_mean_absolute_error: 0.0053
Epoch 5/5
10010/10010 [==============================] - 19s 2ms/step - loss: 1.2726e-04 - mean_absolute_error: 0.0033 - val_loss: 5.2232e-04 - val_mean_absolute_error: 0.0031


In [128]:
s = m.predict(X_train[540, :, :, np.newaxis].reshape(1, 100, 100, 1))

In [129]:
plt.imshow(s[0, :, :, 0])


In [130]:
plt.figure()
plt.imshow(X_train[540, :, :])

In [131]:
v = m.layers[0].weights[0]


In [114]:
from keras.backend import get_session

In [132]:
v_ = v.eval(get_session())

In [144]:
print(v_[:, :, 0, 0])
plt.imshow(v_[:, :, 0, 0])
plt.colorbar()

[[ 0.03331073  0.24320373  0.08035007]
 [ 0.04181295 -0.34073147  0.291245  ]
 [ 0.06266355  0.13147485  0.01135006]]


In [145]:
print(v_[:, :, 0, 1])
plt.imshow(v_[:, :, 0, 1])
plt.colorbar()

[[0.02140622 0.15627892 0.05162006]
 [0.0269519  1.1345521  0.1871164 ]
 [0.04024982 0.08453029 0.00729506]]


In [139]:
q = m.layers[1].weights[0]
q_ = q.eval(get_session())

In [141]:
q_

array([[-1.1652563],
       [ 1.81402  ]], dtype=float32)

In [149]:
scipy.signal.convolve2d(np.array([[1, 1, 1], [0, 0, 1], [1, 1, 0]]), v_[:, :, 0, 0],  mode='same')

array([[-0.29891852,  0.02563722,  0.19371727],
       [ 0.47065286,  0.57085521, -0.11755649],
       [-0.29891852,  0.01317709,  0.42271987]])

In [150]:
scipy.signal.convolve2d(np.array([[1, 1, 1], [0, 0, 1], [1, 1, 0]]), v_[:, :, 0, 1],  mode='same')

array([[1.16150402, 1.37002664, 1.47794744],
       [0.30246525, 0.36692606, 1.27799753],
       [1.16150402, 1.36191835, 0.27164669]])